In [2]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding,LSTM
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import load_model
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_word_list

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [20]:
dataset = pd.read_excel('../input/sentiment/sentiment_analysis.xlsx' , sheet_name = 'Sheet1')
dataset.head()

,Text,Sentiment
0,bana beklediğim cevapları vermiyorsun,0
1,senden istediğim cevaplar bunlar değil,0
2,verdiğin yanıtlar doğru değil,0
3,duymak istediğim cevaplar bunlar değil,0
4,seni seviyorum bro,1


In [22]:
#verimizde bulunan noktalama işaretlerinin temizlenme işlemi
dataset['Text'] = dataset['Text'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))
#büyük harflerin küçük harfe çevrilmesi
dataset['Text'] = dataset['Text'].apply(lambda x: x.lower())
#fazladan boşlukların temizlenmesi
dataset['Text'] = dataset['Text'].apply(lambda x: x.strip())
#cümleler içerisinde bulunan stopword'lerin kaldırılması
def token(values):
    words = nltk.tokenize.word_tokenize(values)
    filtered_words = [word for word in words if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc
dataset['Text'] = dataset['Text'].apply(lambda x: token(x))

In [23]:
dataset['Text']

0             bana beklediğim cevapları vermiyorsun
1            senden istediğim cevaplar bunlar değil
2                     verdiğin yanıtlar doğru değil
3            duymak istediğim cevaplar bunlar değil
4                                seni seviyorum bro
                            ...                    
19018    j7 pro cihazı geldi fakat faturası gelmedi
19019         müşteri hizmetlerine ulaşamama sorunu
19020                           para i̇adesi sorunu
19021                                 mağdur ediyor
19022        ürünü aldığı halde parayı i̇ade etmedi
Name: Text, Length: 19023, dtype: object

In [24]:
data = dataset['Text'].values.tolist()
sentiment = dataset['Sentiment'].values.tolist()
x_train, x_test, y_train, y_test = train_test_split(data,sentiment,test_size = 0.2, random_state = 42)

In [25]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(data)
tokenizer.word_index

{'bir': 1,
 'ürün': 2,
 'iyi': 3,
 'güzel': 4,
 'tavsiye': 5,
 'gayet': 6,
 'ederim': 7,
 'hızlı': 8,
 'aldım': 9,
 'yok': 10,
 'telefon': 11,
 'olarak': 12,
 'ürünü': 13,
 'göre': 14,
 'kadar': 15,
 '2': 16,
 'var': 17,
 'teşekkürler': 18,
 'elime': 19,
 'fiyat': 20,
 'uygun': 21,
 'kullanışlı': 22,
 'fiyata': 23,
 'ben': 24,
 'gün': 25,
 'değil': 26,
 'kargo': 27,
 'biraz': 28,
 '1': 29,
 'kaliteli': 30,
 'kullanıyorum': 31,
 'kalitesi': 32,
 'geldi': 33,
 'gerçekten': 34,
 'tek': 35,
 'şarj': 36,
 '3': 37,
 'sonra': 38,
 'bi': 39,
 'olması': 40,
 'cok': 41,
 'bence': 42,
 'küçük': 43,
 'kesinlikle': 44,
 'önce': 45,
 'tam': 46,
 'oldu': 47,
 'performans': 48,
 'ses': 49,
 'ulaştı': 50,
 'mükemmel': 51,
 'sorun': 52,
 'fakat': 53,
 'oldukça': 54,
 'fazla': 55,
 'ancak': 56,
 'bile': 57,
 'fiyatına': 58,
 'memnun': 59,
 'hepsiburada': 60,
 'başarılı': 61,
 'şık': 62,
 'fiyatı': 63,
 'uzun': 64,
 'telefonu': 65,
 'memnunum': 66,
 'hızı': 67,
 'ayrıca': 68,
 'sipariş': 69,
 'kolay': 70,

In [26]:
#her bir yorumu aynı boyuta getirmek gerekiyor RNN böyle çalışıyor.
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [27]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [28]:
max_tokens=np.mean(num_tokens) + 2 * np.std(num_tokens)

In [29]:
max_tokens=int(max_tokens)

In [30]:
np.sum(num_tokens < max_tokens) /len(num_tokens)

0.9599432266204069

In [33]:
#veriler belirlenen token sayısına göre ayarlanır
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [34]:
x_train_pad[3027]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  545,   10, 1834,   15, 9899, 3943,  228, 1817,  195,
        112, 6740,  179,  690,  644,   55,   10, 2689], dtype=int32)

In [35]:
#tokenlaştırılan kelimeler tekrar string hale geitirilmek için bir fonksiyon yazılması gerekiyor.
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))
#tokenlaştırılan cümleyi tekrar string hale getirmek
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [53]:
#ardışık bir model
model = Sequential()

#her kelimeye karşılık gelen 50 uzunluğunda bir vektör oluşturulur. (Embedding matrisi)
embedding_size = 50

#matris kelime sayısı ve embedding büyüklüğünde olacak, yani 10bine 50 uzunluğunda. Buna da bir isim veriliyor name değişkeniyle.
model.add(Embedding(input_dim=10000,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

#LSTM layerlerinin eklenmesi
## 16 nöronlu LSTM (16 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(LSTM(units=16, return_sequences=True))
## 8 nöronlu LSTM (8 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(LSTM(units=8, return_sequences=True))
## 4 nöronlu LSTM (4 outputlu , return_sequences=False yani default değer, tek bir output verecek)
model.add(LSTM(units=4))
## output layer'ı , görsel olarak gösterilirken dense layer kullanılır.  Tek bir nörondan oluştuğu için 1 yazılır.
model.add(Dense(1, activation='sigmoid'))

#optimizasyon algoritması, 1e-3 = 0.001 demek.
optimizer = Adam(lr=1e-3)

#modeli derlemek, loss fonksiyonu binary_crossentropy -> sadece 2 sınıf ama daha fazla sınıflar için categorical_crossentropy kullanılır.
#metrics -> modelin başarısını görmek için.
# success
model.compile("adam", "binary_crossentropy",
                metrics=['accuracy','binary_crossentropy',tf.keras.metrics.AUC(curve='ROC')], )

In [54]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 52, 50)            500000    
_________________________________________________________________
lstm_39 (LSTM)               (None, 52, 16)            4288      
_________________________________________________________________
lstm_40 (LSTM)               (None, 52, 8)             800       
_________________________________________________________________
lstm_41 (LSTM)               (None, 4)                 208       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 5         
Total params: 505,301
Trainable params: 505,301
Non-trainable params: 0
_________________________________________________________________


In [71]:
 y_train = np.array(y_train)
history = model.fit(x_train_pad, y_train,validation_split=0.25, epochs=5, batch_size=256)


2022-10-24 13:44:04.314593: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
45/45 [==============================] - 26s 322ms/step - loss: 0.6234 - accuracy: 0.7133 - binary_crossentropy: 0.6234 - auc_1: 0.4916 - val_loss: 0.5366 - val_accuracy: 0.7180 - val_binary_crossentropy: 0.5366 - val_auc_1: 0.5709
Epoch 2/5
45/45 [==============================] - 13s 299ms/step - loss: 0.4762 - accuracy: 0.8096 - binary_crossentropy: 0.4762 - auc_1: 0.6712 - val_loss: 0.4280 - val_accuracy: 0.8436 - val_binary_crossentropy: 0.4280 - val_auc_1: 0.7361
Epoch 3/5
45/45 [==============================] - 13s 292ms/step - loss: 0.3503 - accuracy: 0.9003 - binary_crossentropy: 0.3503 - auc_1: 0.7859 - val_loss: 0.3270 - val_accuracy: 0.9117 - val_binary_crossentropy: 0.3270 - val_auc_1: 0.8281
Epoch 4/5
45/45 [==============================] - 13s 285ms/step - loss: 0.2653 - accuracy: 0.9341 - binary_crossentropy: 0.2653 - auc_1: 0.8570 - val_loss: 0.2930 - val_accuracy: 0.9183 - val_binary_crossentropy: 0.2930 - val_auc_1: 0.8776
Epoch 5/5
45/45 [===============

In [75]:
x_test_pad = np.array(x_test_pad)
result = model.evaluate(x_test_pad, y_test)

119/119 [==============================] - 5s 40ms/step - loss: 0.2716 - accuracy: 0.9054 - binary_crossentropy: 0.2716 - auc_1: 0.9088
